<h1>Kalman Filter Experiments</h1>

In [42]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

<h2>Creating the data</h2>

First of all a dataset has to be created. Below a dataset was constructed where a vehicle drives a simple track. The driven track and the development of the x and y coordinate are shown.

In [43]:
positionGroundTruthX = np.array([0, 1, 3, 6, 9, 13, 15, 17, 19, 20, 22, 24, 25, 26, 26, 27, 28, 29, 28, 27, 26, 26, 27, 28, 29, 32, 35, 38, 40, 41, 42])
positionGroundTruthY = np.array([0, 0, 0, 0, 1, 3, 4, 6, 7, 7, 7, 6, 5, 5, 5, 5, 5, 6, 7, 9, 10, 12, 13, 14, 17, 19, 20, 20, 19, 18, 16])
timeAtPosition = np.arange(0, 15.5, 0.5)

# x, y position map
p = figure(title="Ground Truth Data", x_axis_label='x Position', y_axis_label='y Position')
p.toolbar_location = None
p.line(positionGroundTruthX, positionGroundTruthY, line_width=2, color='red')
show(p)

# x position over time
p = figure(title="x Position", x_axis_label='Time in seconds', y_axis_label='x Position')
p.toolbar_location = None
p.line(timeAtPosition, positionGroundTruthX, line_width=2, color='red')
show(p)

# y position over time
p = figure(title="y Position", x_axis_label='Time in seconds', y_axis_label='y Position')
p.toolbar_location = None
p.line(timeAtPosition, positionGroundTruthY, line_width=2, color='red')
show(p)

In order to test the Kalman Filter, sensor data has to be generated. Since there is no access to the simulation environment right now, the sensor data will be the ground truth data multiplied with some gaussian noise. Since the random seed is not set, the data will be different every time this cell is run.

In [44]:
# IMU position data
noiseX = np.random.normal(1, 0.03, 31)
noiseY = np.random.normal(1, 0.03, 31)
positionIMUX = np.multiply(positionGroundTruthX, noiseX)
positionIMUY = np.multiply(positionGroundTruthY, noiseY)

# GNSS position data
noiseX = np.random.normal(1, 0.03, 31)
noiseY = np.random.normal(1, 0.03, 31)
positionGNSSX = np.multiply(positionGroundTruthX, noiseX)
positionGNSSY = np.multiply(positionGroundTruthY, noiseY)


p = figure(title="Comparison of Ground Truth to IMU and GNSS data", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')

show(p)

<h2>Implement the Kalman Filter</h2>

The Kalman Filter will be implemented as described in [An Introduction to the Kalman Filter](http://www.cs.unc.edu/~welch/media/pdf/kalman_intro.pdf).

In [45]:
class KalmanFilter:

    def __init__(self, dt, A, H, Q, R, P):
        self.A = A
        self.H = H
        self.Q = Q
        self.R = R
        self.P0 = P
        self.m = np.shape(H)[0]
        self.n = np.shape(A)[0]
        self.dt = dt
        self.initialized = False
        self.I = np.identity(self.n)
        self.x_hat = np.zeros(self.n)
        self.x_hat_new = np.zeros(self.n)

    def init(self, t0=None, x0=None):
        if x0 is not None:
            self.x_hat = x0
        self.P = self.P0
        self.t0 = 0
        if t0 is not None:
            self.t0 = t0
        self.t = self.t0
        self.initialized = True

    def update(self, y, dt=None, A=None):
        if A is not None:
            self.A = A
        if dt is not None:
            self.dt = dt
        
        if not self.initialized:
            print("Filter not initialized")

        self.x_hat_new = self.A @ self.x_hat
        self.P = self.A @ self.P @ np.transpose(self.A) + self.Q
        self.K = self.P @ np.transpose(self.H) @ np.linalg.inv(self.H @ self.P @ np.transpose(self.H) + self.R)
        self.x_hat_new = self.K @ (y - self.H @ self.x_hat_new)
        self.P = (self.I - self.K @ self.H) @ self.P
        self.x_hat = self.x_hat_new

        self.t = self.t + self.dt

A refers to the system dynamics matrix <br>
H refers to the Output matrix and transforms the measurement to the state vector<br>
Q refers to the process noise covariance and tells us how instable the process is<br>
R refers to the measurement noise covariance and tells us how accurate the measured data is and how much we should trust it<br>
P refers to the estimate error covariance and shows how much the estimates of the filter are off. It converges over time.<br>

In [46]:
n = 2 # number of states
m = 4 # number of measurements
dt = 0.5 # time step

# list of all values to plot
x_hatXValues = [0]
x_hatYValues = [0]

# Model
A = np.array([[0.01, 0], [0, 0.01]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[50, 0], [0, 50]])
R = np.array([[0.001, 0, 0, 0], [0, 0.001, 0, 0], [0, 0, 0.001, 0], [0, 0, 0, 0.001]])
P = np.array([[1, 0], [0, 1]])

filter = KalmanFilter(dt, A, H, Q, R, P)
filter.init()

#print("t=" + str(filter.t) + ", " + "x_hat[0]: " + str(filter.x_hat))
for xIMU, yIMU, xGNSS, yGNSS in zip(positionIMUX, positionIMUY, positionGNSSX, positionGNSSY):
    y = np.array([xIMU, yIMU, xGNSS, yGNSS])
    filter.update(y)
    x_hatXValues.append(filter.x_hat[0])
    x_hatYValues.append(filter.x_hat[1])
    #print("t=" + str(filter.t))
    #print("y=" + str(y))
    #print()
    #print("x_hat=" + str(filter.x_hat))
    #print()

Visualize the Output of the Filter.

In [47]:
p = figure(title="Output of the Kalman Filter", x_axis_label='x Position', y_axis_label='y Position')

# filter output
p.line(x_hatXValues, x_hatYValues, line_width=2, color='orange')

show(p)

Compare the Filter to the Ground Truth

In [48]:
p = figure(title="Comparison of Filter and Ground Truth", x_axis_label='x Position', y_axis_label='y Position')

# filter output
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')
# ground truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label="Ground Truth", line_width=2, color='red')

show(p)

Compare the filter to the ground truth with the sensor data

In [49]:
p = figure(title="Comparison of Ground Truth to IMU and GNSS data", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

<h2>Understanding the Parameters</h2>